In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os

In [23]:
outliers = pd.read_csv("outliers.csv")
outliers = outliers.drop(["Unnamed: 0", "Unnamed: 0.1"], 1)
print(sum(outliers["outliers"]))
outliers.head()

147


,subject,task,sensor,start,stop,length,outliers
0,PDMotion_0020_Test,PS,right_1,11.4125,55.5750,44.1625,True
1,PDMotion_0020_Test,PS,right_2,11.3750,55.6250,44.2500,True
2,PDMotion_0020_Test,PS,left_1,65.9625,76.0125,10.0500,False
3,PDMotion_0020_Test,PS,left_2,66.5750,75.9625,9.3875,False
4,PDMotion_0020_Test,HM,right_1,15.0500,19.8875,4.8375,False


In [17]:
#less than 10 seconds
extended = []
for i, r in outliers.iterrows():
    if r["stop"] - r["start"] < 10:
        extended.append(r["start"] + 10)
    else:
        extended.append(-1)
outliers["extended"] = extended
print(np.sum(outliers["extended"].count()))
outliers.head()


944


,subject,task,sensor,start,stop,length,outliers,extended,continuous
0,PDMotion_0020_Test,PS,right_1,11.4125,55.5750,44.1625,True,-1.000,
1,PDMotion_0020_Test,PS,right_2,11.3750,55.6250,44.2500,True,-1.000,
2,PDMotion_0020_Test,PS,left_1,65.9625,76.0125,10.0500,False,-1.000,NaN
3,PDMotion_0020_Test,PS,left_2,66.5750,75.9625,9.3875,False,76.575,NaN
4,PDMotion_0020_Test,HM,right_1,15.0500,19.8875,4.8375,False,25.050,NaN


In [18]:
#Set up for visual continuous check
continuous = []
for i, r in outliers.iterrows():
    if r["outliers"]:
        continuous.append('')
    else:
        continuous.append(-1)
outliers["continuous"] = continuous
print(np.sum(outliers["continuous"].count()))
outliers.head()

#-1: already valid, between 10 and 20s
#2: longer than 20 but continuous, use stop column
#3: longer than 20, not continuous, but clear outlying spike need to keep going
#4: longer than 20, not continuous, unclear period

944


,subject,task,sensor,start,stop,length,outliers,extended,continuous
0,PDMotion_0020_Test,PS,right_1,11.4125,55.5750,44.1625,True,-1.000,
1,PDMotion_0020_Test,PS,right_2,11.3750,55.6250,44.2500,True,-1.000,
2,PDMotion_0020_Test,PS,left_1,65.9625,76.0125,10.0500,False,-1.000,-1
3,PDMotion_0020_Test,PS,left_2,66.5750,75.9625,9.3875,False,76.575,-1
4,PDMotion_0020_Test,HM,right_1,15.0500,19.8875,4.8375,False,25.050,-1


In [93]:
oc = pd.read_csv("outliers_continuous.csv")
oc = oc.drop(["Unnamed: 0", "Unnamed: 10"], axis=1)
oc.head()

,subject,task,sensor,start,stop,length,outliers,extended,continuous
0,PDMotion_0020_Test,PS,right_1,11.4125,55.5750,44.1625,True,-1.000,3
1,PDMotion_0020_Test,PS,right_2,11.3750,55.6250,44.2500,True,-1.000,3
2,PDMotion_0020_Test,PS,left_1,65.9625,76.0125,10.0500,False,-1.000,-1
3,PDMotion_0020_Test,PS,left_2,66.5750,75.9625,9.3875,False,76.575,-1
4,PDMotion_0020_Test,HM,right_1,15.0500,19.8875,4.8375,False,25.050,-1


In [94]:
oc.head()

,subject,task,sensor,start,stop,length,outliers,extended,continuous
0,PDMotion_0020_Test,PS,right_1,11.4125,55.5750,44.1625,True,-1.000,3
1,PDMotion_0020_Test,PS,right_2,11.3750,55.6250,44.2500,True,-1.000,3
2,PDMotion_0020_Test,PS,left_1,65.9625,76.0125,10.0500,False,-1.000,-1
3,PDMotion_0020_Test,PS,left_2,66.5750,75.9625,9.3875,False,76.575,-1
4,PDMotion_0020_Test,HM,right_1,15.0500,19.8875,4.8375,False,25.050,-1


In [95]:
#set up fixed outliers for 2s
fixed = []
final_start = []
final_stop = []

for i, r in oc.iterrows():
    if float(r["continuous"]) == 2:
        fixed.append("continuous")
        final_start.append(r["start"])
        final_stop.append(r["stop"])
    elif float(r["extended"]) != -1:
        fixed.append("extended")
        final_start.append(r["start"])
        final_stop.append(r["extended"])
    elif float(r["continuous"]) == -1:
        fixed.append("regular")
        final_start.append(r["start"])
        final_stop.append(r["stop"])
    else:
        fixed.append(False)
        final_start.append(-1)
        final_stop.append(-1)

print(len(fixed), len(final_start), len(final_stop))

oc["fixed"] = fixed
oc["final_start"] = final_start
oc["final_stop"] = final_stop



944 944 944


In [96]:
#Check for pairs
for i, r in oc.iterrows():
    if not r["fixed"]:
        if r["continuous"] == 3 or r["continuous"] == 4:
            if r["sensor"] == "right_1" or r["sensor"] == "left_1":
                r2 = oc.loc[i+1]
                if r2["fixed"]:
                    oc.loc[i, "fixed"] = "pair"
                    oc.loc[i, "final_start"] = r2["final_start"]
                    oc.loc[i, "final_stop"] = r2["final_stop"]
            else:
                r2 = oc.loc[i-1]
                if r2["fixed"]:
                    oc.loc[i, "fixed"] = "pair"
                    oc.loc[i, "final_start"] = r2["final_start"]
                    oc.loc[i, "final_stop"] = r2["final_stop"]
oc.head()                    

,subject,task,sensor,start,stop,length,outliers,extended,continuous,fixed,final_start,final_stop
0,PDMotion_0020_Test,PS,right_1,11.4125,55.5750,44.1625,True,-1.000,3,False,-1.0000,-1.0000
1,PDMotion_0020_Test,PS,right_2,11.3750,55.6250,44.2500,True,-1.000,3,False,-1.0000,-1.0000
2,PDMotion_0020_Test,PS,left_1,65.9625,76.0125,10.0500,False,-1.000,-1,regular,65.9625,76.0125
3,PDMotion_0020_Test,PS,left_2,66.5750,75.9625,9.3875,False,76.575,-1,extended,66.5750,76.5750
4,PDMotion_0020_Test,HM,right_1,15.0500,19.8875,4.8375,False,25.050,-1,extended,15.0500,25.0500


In [100]:
td = 0
for i, r in oc.iterrows():
    if r["fixed"] == False:
        td+=1
print(td)

#146->133->70
#

0


In [68]:
oc.to_csv("outliers_fixed.csv")

In [98]:
#fix other cases with largest continous chunk
chunk_start = []
chunk_end = []

for i, r in oc.iterrows():
    if not r["fixed"]:
        s_data = pd.read_csv("series/"+r["subject"]+"_"+r["task"]+"_"+r["sensor"]+".csv")
        s_data = s_data[["relative_time", "threshold"]].dropna()
        fixed_start, fixed_stop = largest_chunk(s_data)
        
        oc.loc[i, "fixed"] = "chunk"
        oc.loc[i, "final_start"] = fixed_start
        oc.loc[i, "final_stop"] = fixed_stop



In [91]:
s_data
s_data.query('threshold == 1' ).head(1).relative_time.values[0]

17.9375

In [92]:
def largest_chunk(s_data, max_time = 2):
    pairs = []
    cstart = 0 #s_data.query('threshold == 1').head(1).relative_time.values[0]
    cend = 0
    for i, r in s_data.iterrows():
        if r["threshold"] == 1:
            if r["relative_time"] - cend > max_time:
                pairs.append((cstart, cend))
                cstart = r["relative_time"]
                cend = r["relative_time"]
            else:
                cend = r["relative_time"]
    
    max_pair = pairs[0]
    max_time = pairs[0][1]-pairs[0][0]
    for p in pairs:
        if p[1]-p[0] > max_time:
            max_pair = p
            max_time = p[1] - p[0]
    return max_pair[0], max_pair[1]


In [99]:
oc.head()

,subject,task,sensor,start,stop,length,outliers,extended,continuous,fixed,final_start,final_stop
0,PDMotion_0020_Test,PS,right_1,11.4125,55.5750,44.1625,True,-1.000,3,chunk,11.4125,16.8125
1,PDMotion_0020_Test,PS,right_2,11.3750,55.6250,44.2500,True,-1.000,3,chunk,11.3750,17.3125
2,PDMotion_0020_Test,PS,left_1,65.9625,76.0125,10.0500,False,-1.000,-1,regular,65.9625,76.0125
3,PDMotion_0020_Test,PS,left_2,66.5750,75.9625,9.3875,False,76.575,-1,extended,66.5750,76.5750
4,PDMotion_0020_Test,HM,right_1,15.0500,19.8875,4.8375,False,25.050,-1,extended,15.0500,25.0500
